In [1]:
import numpy as np
import pandas as pd
import math
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import multinomial
from typing import List
import seaborn as sns

# system characterisation

class State:
    def __init__(self, mean, eta, lambda_switch, sigma):
        self.m = mean
        self.e = eta
        self.l = lambda_switch
        self.s = sigma


M = 4
N = 4

state_1 = State(2.5, 0.05, 0.05, 0.05)
state_2 = State(4,0.1,0.15,0.05)


def Profit(E, P, vc, fc, pi_tax, c_tax):
    pi = ((E * (P - vc - c_tax)) - fc) * (1 - pi_tax)
    return pi


T = 2
E = 20000
r = 0.0125
vc = 1
fc = 0
pi_tax = 0.2
c_tax = 0.4

# finite difference parameter definition ##

# parameter defn for P_i in i = 2,...,i_max-1 #

def difference(P, p_low, p_high, sigma, eta, mean):
    difference = np.zeros(2)
    a_i_central = (sigma**2)*(P**2)/((P - p_low)*(p_high - p_low)) - (eta*(mean - P))/(p_high - p_low)
    b_i_central = (sigma**2)*(P**2)/((p_high - P)*(p_high - p_low)) + (eta*(mean - P))/(p_high - p_low)
    a_i_forward = (sigma**2)*(P**2)/((P - p_low)*(p_high - p_low))
    b_i_forward = (sigma**2)*(P**2)/(p_high - P)*(p_high - p_low) + (eta*(mean - P))/(p_high - p_low)
    a_i_backward = (sigma**2)*(P**2)/((P - p_low)*(p_high - p_low)) - (eta*(mean - P))/(p_high - p_low)
    b_i_backward = (sigma**2)*(P**2)/((p_high - P)*(p_high - p_low))
    if a_i_central > 0:
        difference[0] = a_i_central
    else:
        difference[0] = a_i_forward
    if b_i_central > 0:
        difference[1] = b_i_central
    else:
        difference[1] = b_i_backward
    return (difference)


# parameter defn for P_i in i = 1 ##

def difference_min(P, p_step, eta, mean):
    a_i = 0
    b_i = eta * (mean - P) / (p_step)
    return [a_i, b_i]


# parameter defn for P_i in i = i_max ##

def difference_max(P, p_step, eta, mean):
    a_i = abs(eta * (mean - P) / (p_step))
    b_i = 0
    return [a_i, b_i]


# lattice parameters ##

s0_step = 1000000

p_1 = 0.5
p_max = 8

s0_1 = 1000000
s0_max = 5000000

# lattice construction #

P_grid = [0.25, 1, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4, 4.25, 4.5, 4.75, 5, 5.25, 5.5, 5.75, 6, 6.5, 7, 8, 11]

s0_grid = list(range(s0_1, s0_max, s0_step))

S = 2
temp_node = np.full((S,len(P_grid), len(s0_grid), M, N), 0)
node_value = np.full((S,len(P_grid), len(s0_grid), M, N), 0)
Lv = np.full((S,len(P_grid), len(s0_grid), M, N), 0)

# here we store the optimal state n in N conditional on prior state m in M ##

temp_path = np.full((S,len(P_grid), len(s0_grid), M), 0)
opt_path = []

cost = np.zeros((4, 4))
temp_compare = np.zeros((4))

## cost of staying in or returning to pre-construction state ##

cost[0, 0] = 0
cost[1, 0] = 1000000
cost[2, 0] = 1000000
cost[3, 0] = 1000000

## cost of staying in or returning to moving to operating state ##

cost[1, 1] = 0
cost[0, 1] = 20000
cost[2, 1] = 2000
cost[3, 1] = 1000000

## cost of staying in or returning to moving to mothball state ##

cost[0, 2] = 1000000
cost[1, 2] = 2000
cost[2, 2] = 0
cost[3, 2] = 1000000

## cost of staying in or returning to moving to abandoned state ##

cost[0, 3] = 1000000
cost[1, 3] = 1000000
cost[2, 3] = 10000
cost[3, 3] = 0

In [2]:
for t in range(10):
    for q in range(4):

## M refers to adjacent state, N refers to current state ##

## Quarterly profit at each node ##
        for s in range(2):
            for i in range(len(P_grid)):
                for k in range(len(s0_grid)):
                    for m in range(M):
                        for n in range(N):
                            if n == 1:
                                if (80 - (t*4 + q))*E < s0_grid[k]:
                                    temp_node[s,i,k,m,n] = temp_node[s,i,k,m,n] + Profit(E,P_grid[i],vc,fc,pi_tax,c_tax)
                                else:
                                    temp_node[s,i,k,m,n] = temp_node[s,i,k,m,n] - fc
                            else:
                                temp_node[s,i,k,m,n] = temp_node[s,i,k,m,n] - fc

## Lagrange Differential at each node ##

## then we loop over lagrange differential for nodes inside the boundary ##
        for s in range(2):
            for i in range(1,len(P_grid) - 2):
                for k in range(len(s0_grid)):
                    for m in range(M):
                        for n in range(N):
                            if s == 0:
                                diff = difference(P_grid[i],P_grid[i-1],P_grid[i+1],state_1.s,state_1.e,state_1.m)
                                Lv[s,i,k,m,n] = diff[0]*temp_node[s,i - 1,k,m,n] + diff[1]*temp_node[s,i + 1,k,m,n] - (diff[0]+diff[1]+r)*temp_node[s,i,k,m,n] + state_1.l*(temp_node[0,i,k,m,n]-temp_node[1,i,k,m,n])
                            else:
                                diff = difference(P_grid[i],P_grid[i-1],P_grid[i+1],state_1.s,state_1.e,state_1.m)
                                Lv[s,i,k,m,n] = diff[0]*temp_node[s,i - 1,k,m,n] + diff[1]*temp_node[s,i + 1,k,m,n] - (diff[0]+diff[1]+r)*temp_node[s,i,k,m,n] + state_2.l*(temp_node[1,i,k,m,n]-temp_node[0,i,k,m,n])

## we loop over Lagrange differential for boundary nodes ##

        for s in range(2):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        if s == 0:
                            diff = difference_min(P_grid[0],(P_grid[1]-P_grid[0]),state_1.e,state_1.m)
                            Lv[s,0,k,m,n] = diff[1]*temp_node[s,1,k,m,n] - (diff[1]+r)*temp_node[s,0,k,m,n] + state_1.l*(temp_node[0,0,k,m,n]-temp_node[1,0,k,m,n])
                        else:
                            diff = difference_min(P_grid[0],(P_grid[1]-P_grid[0]),state_2.e,state_2.m)
                            Lv[s,0,k,m,n] = diff[1]*temp_node[s,1,k,m,n] - (diff[1]+r)*temp_node[s,0,k,m,n] + state_2.l*(temp_node[1,0,k,m,n]-temp_node[0,0,k,m,n])

        for s in range(2):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        if s == 0:
                            diff = difference_max(P_grid[len(P_grid)-1],(P_grid[len(P_grid)-1]-P_grid[len(P_grid)-2]),state_1.e,state_1.m)
                            Lv[s,len(P_grid) - 1,k,m,n] = diff[0]*temp_node[s,len(P_grid)-2,k,m,n] - (diff[0]+r)*temp_node[s,len(P_grid)-1,k,m,n] + state_1.l*(temp_node[0,len(P_grid) - 1,k,m,n]-temp_node[1,len(P_grid) - 1,k,m,n])
                        else:
                            diff = difference_max(P_grid[len(P_grid)-1],(P_grid[len(P_grid)-1]-P_grid[len(P_grid)-2]),state_2.e,state_2.m)
                            Lv[s,len(P_grid) - 1,k,m,n] = diff[0]*temp_node[s,len(P_grid)-2,k,m,n] - (diff[0]+r)*temp_node[s,len(P_grid)-1,k,m,n] + state_2.l*(temp_node[1,len(P_grid) - 1,k,m,n]-temp_node[0,len(P_grid) - 1,k,m,n])

## then we apply Lv to temporary node values ##
        for s in range(2):
            for i in range(len(P_grid)):
                for k in range(len(s0_grid)):
                    for m in range(M):
                        for n in range(N):
                            temp_node[s,i,k,m,n] = temp_node[s,i,k,m,n] + Lv[s,i,k,m,n]


## we apply time constraints with arbitrarily high costs ##
    
    if t < (10 - 3):
        cost[3,3] = 1000000
    if t < (10 - 2):
        cost[2,2] = 1000000
        cost[2,1] = 1000000
    if t < (10 - 1):
        cost[1,1] = 1000000

## we apply conditional costs on each state ##   
    
    for s in range(2):
        for i in range(len(P_grid)):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        temp_node[s,i,k,m,n] = temp_node[s,i,k,m,n] - cost[m,n]

## we identify the optimal conditional state n in N for every adjacent state m in M ##

    for s in range(2):
        for i in range(len(P_grid)):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        temp_compare[n] = temp_node[s,i,k,m,n]
                    temp_path[s,i,k,m] = np.argmax(temp_compare)

## we for the next cycle, we adjust option value ##

    for s in range(2):
        for i in range(len(P_grid)):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        node_value[s,i,k,m,n] = temp_node[s,i,k,m,(temp_path[s,i,k,m])]
                        temp_node[s,i,k,m,n] = node_value[s,i,k,m,n]

In [11]:
print(temp_node[1,:,:,0,0])

[[    476   70903   70903   70903]
 [    960   70895   70895   70895]
 [  10981  152909  152909  152909]
 [  45101  264250  264250  264250]
 [  70092  327965  327965  327965]
 [  94863  386021  386021  386021]
 [ 120000  442595  442595  442595]
 [ 145566  498620  498620  498620]
 [ 171488  554398  554398  554398]
 [ 197655  610050  610050  610050]
 [ 223966  665632  665632  665632]
 [ 250351  721173  721173  721173]
 [ 276769  776689  776689  776689]
 [ 303199  832187  832187  832187]
 [ 329633  887674  887674  887674]
 [ 356068  943155  943155  943155]
 [ 382503  998631  998631  998631]
 [ 408937 1054119 1054119 1054119]
 [ 435376 1109606 1109606 1109606]
 [ 461814 1165196 1165196 1165196]
 [ 515754 1288937 1288937 1288937]
 [ 630401 1924282 1924282 1924282]
 [ 890400 4024000 4024000 4024000]
 [ 959859 3777072 3777072 3777072]]
